# ADK Web Help

In [ ]:
!adk web --help


# To run the Agent in ADK Web with DEBUG

### You can use any log level such as INFO , CRITICAL , ERROR , WARNING

In [ ]:
!adk web --log_level DEBUG

## To Evaluate the Agent. Run this in Terminal from Root Directory

In [ ]:
!adk eval src/mediflow_ai tests/mediflow_evalset.json --config_file_path tests/test_config.json --print_detailed_results